In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ase
import rmsd

In [3]:
from ase.visualize import view
from ase.optimize import BFGS
from ase import units
from ase.io.trajectory import Trajectory
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet
from ase.md.nvtberendsen import NVTBerendsen

In [4]:
from qml.kernels import get_atomic_local_kernel
from qml.kernels import get_atomic_local_gradient_kernel
from qml.math import svd_solve

In [5]:
# import path
import sys
sys.path.append("tutorial") # go to parent dir

In [6]:
# tutorial modules
import training
import calculators

## Load in a molcule

Let us start with importing a molecule

In [7]:
atom_labels, coordinates = rmsd.get_coordinates_xyz("examples/ethanol.xyz")
molecule = ase.Atoms(atom_labels, coordinates)
view(molecule, viewer="x3d")

This molecule looks weird and not very optimized. 

## Making a predictor

To interact with the molecule we want to make a ASE Calculator class and this needs a way to predict energies and forces. We want it fast, so we will create a QML Model for this.


To train a model we need energies and forces from similar chemical space, and as a good chef we have already prepared that. In the `data` folder there is training data for small organic molecules of which we select a few. These contains properties include coordinates, energy and the force vectors. 

We use this to train a KRR model, of which we can feed to a ASE Calculator.


In [8]:
# Select chemical space
filenames = [
    "mc_amon00000_001_C.csv",
    "mc_amon00001_001_O.csv",
    "mc_amon00003_001_C=O.csv",
    "mc_amon00005_001_CO.csv",
    "mc_amon00007_001_CC.csv",
    "mc_amon00011_001_C=C.csv",
    "mc_amon00012_001_CC=O.csv",
    "mc_amon00016_001_C=CO.csv",
    "mc_amon00024_001_COC.csv",
    "mc_amon00030_001_CCO.csv",
]
directory = "data/training_data/"
filenames = [directory + f for f in filenames]

In [9]:
# Define representation parameters

# no offset needed
offset = 0.0

# The cut distances
distance_cut = 6.0

# Maximum size of representation vector (max molecule size)
max_atoms = 25

# Which elements are included in the presentation
elements = [1, 6, 8]


parameters = {
    "pad": max_atoms,
    "rcut": distance_cut,
    "acut": distance_cut,
    "elements": elements,
}


In [10]:
# We choose a random number of training examples and hyper-parameter
# assumeing that we already optimized it
n_train = 150
sigma = 10.0

In [11]:
# Calling a func for reading in the specific dataset and 
# calculates the representations and derivative of the representation
representations, d_representations, charges, energies, forces = training.read_csv_files(filenames, parameters=parameters)

In [12]:
# Calculate the alpha and then we have a model

n_points = len(energies)

indexes = list(range(n_points))
np.random.shuffle(indexes)
training_indexes = indexes[:n_train]

training_repr = representations[training_indexes]
training_d_repr = d_representations[training_indexes]
training_charges = [charges[i] for i in training_indexes]
training_energies = energies[training_indexes]
training_forces = [forces[i] for i in training_indexes]
training_forces = np.concatenate(training_forces)


kernel_te = get_atomic_local_kernel(
    training_repr,
    training_repr,
    training_charges,
    training_charges,
    sigma)


kernel_tf = get_atomic_local_gradient_kernel(
    training_repr,
    training_repr,
    training_d_repr,
    training_charges,
    training_charges,
    sigma)

C = np.concatenate((kernel_te, kernel_tf))
Y = np.concatenate((training_energies, training_forces.flatten()))

alpha = svd_solve(C, Y, rcond=1e-11)

## Optimization using OQML forces

Doesn't seem very optimized, so let us optimize it. We do this by creating a ASE Calculator class

In [13]:
cal_parameters = {}
cal_parameters["offset"] = offset
cal_parameters["sigma"] = sigma
cal_parameters["representation_parameters"] = parameters

calculator = calculators.FCHL19Calculator(
    cal_parameters,
    training_repr,
    training_charges,
    alpha)

In [14]:
molecule.set_calculator(calculator)

In [15]:
dyn = BFGS(molecule)
dyn.run(fmax=0.1)

      Step     Time          Energy         fmax
BFGS:    0 15:06:14      -34.842546        1.1038
BFGS:    1 15:06:14      -34.865248        0.8136
BFGS:    2 15:06:14      -34.878755        0.4558
BFGS:    3 15:06:14      -34.889120        0.2546
BFGS:    4 15:06:14      -34.892467        0.2559
BFGS:    5 15:06:14      -34.904473        0.4374
BFGS:    6 15:06:14      -34.910448        0.3095
BFGS:    7 15:06:14      -34.920175        0.2642
BFGS:    8 15:06:14      -34.926546        0.2661
BFGS:    9 15:06:14      -34.936284        0.2726
BFGS:   10 15:06:14      -34.943258        0.2751
BFGS:   11 15:06:14      -34.946642        0.4398
BFGS:   12 15:06:15      -34.951179        0.1868
BFGS:   13 15:06:15      -34.954267        0.2116
BFGS:   14 15:06:15      -34.958149        0.2239
BFGS:   15 15:06:15      -34.960748        0.1627
BFGS:   16 15:06:15      -34.962811        0.1243
BFGS:   17 15:06:15      -34.964828        0.1291
BFGS:   18 15:06:15      -34.966706        0.1051
B

True

In [16]:
view(molecule, viewer="x3d")

## Running a molecular dynamics

with the same calculator we can also do other type of dynamics, such as molecular dynamics.

In [17]:
time = 0.5
temp = 150
MaxwellBoltzmannDistribution(molecule, 200 * units.kB)
dyn = NVTBerendsen(molecule, time*units.fs, temp, time*units.fs)

In [18]:
traj = Trajectory('_tmp_traj.traj', 'w', molecule)
dyn.attach(traj.write, interval=10)

In [19]:
dyn.run(1000)
traj.close()

In [20]:
traj = Trajectory('_tmp_traj.traj', mode='r')
view(traj, viewer="ngl")
#help(traj)

_ColormakerRegistry()